In [ ]:
import rasterio
import numpy as np
from sklearn.decomposition import PCA

# Process the image into principal components

def read_raster(filepath):
    """Reads a multi-band raster file and returns the data and metadata."""
    with rasterio.open(filepath) as src:
        data = src.read()
        metadata = src.meta
    return data, metadata

def write_raster(filepath, data, metadata):
    """Writes a multi-band raster file."""
    metadata.update({"count": data.shape[0], "dtype": data.dtype.name})
    with rasterio.open(filepath, "w", **metadata) as dst:
        dst.write(data)

def apply_pca(image_data, n_components=3):
    """Applies PCA to multi-band image data."""
    # Reshape image data to 2D (bands, pixels)
    bands, height, width = image_data.shape
    reshaped_data = image_data.reshape(bands, -1).T

    # Perform PCA
    pca = PCA(n_components=n_components)
    pca_result = pca.fit_transform(reshaped_data)

    # Reshape the PCA result back to (components, height, width)
    pca_image = pca_result.T.reshape(n_components, height, width)

    return pca_image

def main(input_filepath, output_filepath):
    """Main function to process the Sentinel-2 image."""
    # Read the Sentinel-2 image
    image_data, metadata = read_raster(input_filepath)

    # Apply PCA to reduce to 3 bands
    pca_image = apply_pca(image_data, n_components=3)

    # Update metadata for 3 bands
    metadata["count"] = 3

    # Write the PCA-transformed image to a new file
    write_raster(output_filepath, pca_image.astype(np.float32), metadata)

if __name__ == "__main__":
    # Input Sentinel-2 image path
    input_filepath  = <"path to your image">
    # Output PCA-transformed image path
    output_filepath = "pca_transformed_image.tif"

    main(input_filepath, output_filepath)


In [ ]:
# Display the image using matplotlib

from skimage.exposure import equalize_hist

def display_image_with_histogram_equalization(filepath):
    """Reads a multi-band satellite image and displays it with histogram equalization applied to each band."""
    with rasterio.open(filepath) as src:
        # Read the image data
        image_data = src.read()
    
    # Normalize each band for display
    normalized_image = np.zeros_like(image_data, dtype=np.float32)
    for i in range(image_data.shape[0]):
        band = image_data[i]
        band_min, band_max = band.min(), band.max()
        normalized_image[i] = (band - band_min) / (band_max - band_min)
    
    # Apply histogram equalization to each normalized band
    equalized_image = np.zeros_like(normalized_image, dtype=np.float32)
    for i in range(normalized_image.shape[0]):
        equalized_image[i] = equalize_hist(normalized_image[i])
    
    # Stack the first three bands into an RGB image for display
    rgb_image = np.dstack([equalized_image[0], equalized_image[1], equalized_image[2]])
    
    # Display the image
    plt.figure(figsize=(10, 10))
    plt.imshow(rgb_image, vmin=0, vmax=1)
    plt.title("Satellite Image with Histogram Equalization (Display Only)")
    plt.axis("off")
    plt.show()
    
# Example usage
satellite_image_path = "pca_transformed_image.tif"  # Replace with your image file path
display_image_with_histogram_equalization(satellite_image_path)